In [74]:
from web_scraper.web_scraper import CompendiumScraper
import pandas as pd
from selenium.webdriver.common.by import By
import psycopg2
import subprocess
import yaml
import sqlalchemy as sa
import time
import re
import requests
import json

from selenium.webdriver.common.action_chains import ActionChains

In [59]:
with open(r'\Users\jasre\Code\ws_config.yml', 'r') as yml:
    config = yaml.safe_load(yml)

# Running To Do
- Parse follow ups for JP versions of characters (manual...)
- Add notes for characters:
  - Deuce -- Her Sforzando buff isn't included in the BT aura and will have do be added in the visual
- Set up marts
- Set up Streamlit
- Fix manual followup DF
  - Laguna's Desperado debuff should be `ally_attack, damaging`, not just `ally_attack`
  - Weiss' standard followup should be `ally_attack, damaging`, not just `ally_attack`
- Add raw source description for `raw_ability_ranks`. 

In [3]:
cs = CompendiumScraper('\\Users\\jasre\\Code\\ws_config.yml')

2023-09-09 07:54:15,187 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:15,187 - web_scraper.web_scraper - Retrieved all main character links. Generating remaining links for each character.
2023-09-09 07:54:15,187 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:15,248 - web_scraper.web_scraper - self.character_dict_omnibus entry for KELGER was successful
2023-09-09 07:54:15,276 - web_scraper.web_scraper - self.character_dict_omnibus entry for JIHLNABAAT was successful
2023-09-09 07:54:15,291 - web_scraper.web_scraper - self.character_dict_omnibus entry for WEISS was successful
2023-09-09 07:54:15,366 - web_scraper.web_scraper - self.character_dict_omnibus entry for ASTOS was successful
2023-09-09 07:54:15,396 - web_scraper.web_scraper - self.character_dict_omnibus entry for IRIS was successful
2023-09-09 07:54:15,433 - web_scraper.web_scraper - self.character_dict_omnibus entry for REKS 

In [4]:
char_name = 'cecilpaladin'

bt_df = cs.retrieve_hp_caps_from_bt(char_name=char_name, JP=True, return_output=True)

bt_df

2023-09-09 07:54:18,627 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:18,627 - web_scraper.web_scraper - Retrieving BT info for CECILPALADIN.
2023-09-09 07:54:18,627 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:31,583 - web_scraper.web_scraper - Standard BT stack slider div-class not found. Trying second possibility.
2023-09-09 07:54:31,599 - web_scraper.web_scraper - No stack slider found. Assuming CECILPALADIN has a BT without stacks.
2023-09-09 07:54:31,672 - web_scraper.web_scraper - Retrieved BT info for CECILPALADIN.


,char_name,bt_personal_hp_dmg_cap_up,bt_party_hp_dmg_cap_up,enemy_count_apply_list,game_version
0,cecilpaladin,0,50,"[1, 2, 3]",JP


In [5]:
ability_df = cs.generate_ability_df(char_name=char_name, JP=True)

2023-09-09 07:54:31,715 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:31,716 - web_scraper.web_scraper - Parsing ability dict to df for CECILPALADIN.
2023-09-09 07:54:31,717 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:31,718 - web_scraper.web_scraper - Couldn't find ability_dict for CECILPALADIN. Creating it now.
2023-09-09 07:54:31,718 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:31,720 - web_scraper.web_scraper - Generating ability dictionary for CECILPALADIN
2023-09-09 07:54:31,721 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:38,065 - web_scraper.web_scraper - This iteration caught 4 abilities.
2023-09-09 07:54:38,065 - web_scraper.web_scraper - -----------
2023-09-09 07:54:39,096 - web_scraper.web_scraper - This iteration caught 4 abilities.
2023-09-09 07:54:39,096 - web_sc

In [6]:
ability_df

,char_name,ability_name,ability_id,main_target_hp_attacks,non_target_hp_attacks,hp_dmg_cap_up_perc,attribute_list,game_version
0,cecilpaladin,Light Slash,17119,1,0,20,"[Melee, Holy]",JP
1,cecilpaladin,Luminous Blast,17121,2,1,20,"[Ranged, Holy]",JP
2,cecilpaladin,Saint's Fall,21415,10,1,20,"[Melee, Holy]",JP
3,cecilpaladin,Holy Flame,21417,6,6,20,"[Group, Ranged, Holy, Heals]",JP
4,cecilpaladin,Light Enchant All,17071,0,0,0,"[Instant, NoSummon, NoEX]",JP
5,cecilpaladin,Paladin Force,21418,8,8,10,"[Melee, Holy]",JP
6,cecilpaladin,Luminous Shard,21419,8,8,15,"[Group, Ranged, Holy, Heals, NoFree]",JP
7,cecilpaladin,Soul Shift,21421,2,2,350,"[Group, Ranged, Holy]",JP
8,cecilpaladin,Spark of Saint,21426,4,4,250,"[Group, Ranged, Heals, NoFree]",JP
9,cecilpaladin,Enhanced Holy Flame (C),11254,1,1,0,"[Group, Ranged, Holy, Heals, Instant, NoEX, No...",JP


In [7]:
ha_df = cs.retrieve_ha_hp_dmg_cap_up(char_name=char_name, JP=True, return_output=True)

ha_df

2023-09-09 07:54:43,846 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:43,846 - web_scraper.web_scraper - Retrieving High Armor info for CECILPALADIN.
2023-09-09 07:54:43,846 - web_scraper.web_scraper - ====================================================
2023-09-09 07:54:54,733 - web_scraper.web_scraper - High armor info parsed for CECILPALADIN.


,char_name,personal_hp_dmg_cap_up,party_ha_hp_dmg_cap_up,game_version
0,cecilpaladin,10,10,JP


# Following can be used to manually upload data to Postgres if the web scraper fails to do so. 
## DO NOT DELETE!

In [ ]:
engine_url = sa.URL.create(
        "postgresql",
        username=config['pg_user'],
        password=config['pg_pass'],
        host=config['pg_host'],
        database=config['pg_db']
    )
    
engine = sa.create_engine(engine_url)

bt_effects = pd.read_csv(config['datasets_dir'] + 'raw_bt_effects.csv')
abilities = pd.read_csv(config['datasets_dir'] + 'raw_abilities.csv')
ha_caps = pd.read_csv(config['datasets_dir'] + 'raw_high_armor_caps.csv')

with engine.begin() as conn:
    abilities.to_sql('raw_abilities', con=conn, index=False, if_exists='append')
    bt_effects.to_sql('raw_bt_effects', con=conn, index=False, if_exists='append')
    ha_caps.to_sql('raw_high_armor_caps', con=conn, index=False, if_exists='append')

# The following cell will upload your manually-entered followups df into Postgres, including with the correct encodings. DO NOT DELETE.

In [64]:
engine_url = sa.URL.create(
        "postgresql",
        username=config['pg_user'],
        password=config['pg_pass'],
        host=config['pg_host'],
        database=config['pg_db']
    )
    
engine = sa.create_engine(engine_url)

followups_manual_entry = pd.read_csv(r'\Users\jasre\Code\dffoo-data-pipeline\datasets\followups_manual_entry.csv', encoding='WINDOWS-1252')

with engine.begin() as conn:
    followups_manual_entry.to_sql('raw_followups_manual_entry', con=conn, index=False, if_exists='append')